In [1]:
# import libraries related to querying links and downloading files from the web
import pandas as pd
from pipmag import file_utils as fu
from pipmag import selector_utils as su
from pipmag import ads_utils as ads
# If you get the error ModuleNotFoundError: No module named 'pipmag', run the following line and restart the kernel:
# %pip install -e ..

In [2]:
# get the latest csv file
latest_updated_la_palma_obs_data_file = '../data/la_palma_obs_data.csv'

In [6]:
# Load the DataFrame from the CSV file
df = pd.read_csv('../data/la_palma_obs_data.csv')

# Convert the strings in 'links', 'video_links', 'image_links', 'instruments' columns back to lists
df['links'] = df['links'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df['video_links'] = df['video_links'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df['image_links'] = df['image_links'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df['instruments'] = df['instruments'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date_time    666 non-null    int64  
 1   date_time.1  666 non-null    object 
 2   year         666 non-null    int64  
 3   month        666 non-null    int64  
 4   day          666 non-null    int64  
 5   time         666 non-null    object 
 6   instruments  666 non-null    object 
 7   target       0 non-null      float64
 8   comments     0 non-null      float64
 9   video_links  666 non-null    object 
 10  image_links  666 non-null    object 
 11  links        666 non-null    object 
 12  num_links    666 non-null    int64  
 13  polarimetry  0 non-null      float64
dtypes: float64(3), int64(5), object(6)
memory usage: 73.0+ KB


In [7]:
# create a widget to display movies based on year, month, day and time
# and to update the target, instrumnets and comments columns of the dataframe
selector = su.VideoSelector2(df, ['target', 'instruments', 'polarimetry', 'comments'])
selector.create_widget()

Dropdown(description='Year:', options=(2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023), valu…

Dropdown(description='Month:', options=(), value=None)

Dropdown(description='Day:', options=(), value=None)

Dropdown(description='Time:', options=(), value=None)

Dropdown(description='Links:', options=(), value=None)

...

Text(value='', description='target:')

Text(value='', description='instruments:')

Text(value='', description='polarimetry:')

Text(value='', description='comments:')

Button(description='Update', style=ButtonStyle())

In [10]:
# 🔍 ADS Search
index = 36
search = ads.ADS_Search(df)
search.get_results(index, pretty_print=True)

In [12]:
# save the updated dataframe as a pickle file
df.to_pickle('../data/'+ fu.add_timestamp('la_palma_obs_data.pkl'))